In [1]:
import copy
import json
import os, glob

from itertools import product

import numpy as np
import pandas as pd

import torch
# %matplotlib inline

import torchaudio

from torchaudio.functional.filtering import lowpass_biquad, highpass_biquad
from torchaudio.transforms import Spectrogram, MelSpectrogram, Resample

from matplotlib import rcParams

from collections import defaultdict

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tqdm import tqdm

from src.model.model import DecoderOnlyNetwork
from src.synth.parameters_normalizer import Normalizer
from src.model.loss import spectral_loss
from synth.synth_architecture import SynthModular
# from config import SynthConfig
from src.main_hp_search_dec_only import configure_experiment

# from config import configure_experiment
from dataset.ai_synth_dataset import AiSynthDataset
from synth.synth_constants import synth_constants

%matplotlib inline
import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt


import seaborn as sns
sns.set_style('whitegrid')

ModuleNotFoundError: No module named 'src'

In [ ]:
# Setup experiment

exp_name = 'training_visualization_fm_saw'
dataset_name = 'fm_saw_single'
config_name = r'C:\Users\noamk\PycharmProjects\ai_synth\configs\lfo_saw_single_synth_config_hp_search.yaml'
device = 'cuda'

cfg = configure_experiment(exp_name, dataset_name, config_name, debug=True)

synth = SynthModular(preset_name=cfg.synth.preset,
                     synth_constants=synth_constants,
                     device=device)

decoder_net = DecoderOnlyNetwork(preset=cfg.synth.preset, device=device)
normalizer = Normalizer(cfg.synth.note_off_time, cfg.synth.signal_duration, synth_constants)

In [ ]:
cfg.multi_spectral_loss_spec_type = 'SPECTROGRAM'
loss_handler = spectral_loss.SpectralLoss(loss_type=cfg.loss.spec_loss_type,
                                          loss_preset=cfg.loss.preset,
                                          synth_constants=synth_constants, device=device)

params_loss_handler = torch.nn.MSELoss()

loss_handler.spectrogram_ops

In [ ]:
dataset = AiSynthDataset(r'C:\Users\noamk\PycharmProjects\ai_synth\data\lfo_saw_single\train', noise_std=0)

target_sample = dataset[0]
target_signal, target_param_dict, signal_index = target_sample

print(f"target parameters full range: \n{target_param_dict}")

target_param_dict = {(1, 1): {'operation': 'lfo',
                            'parameters': {'active': torch.tensor([[-1000.0]], device=device),
                                           'output': [[(-1, -1)]],
                                           'freq': torch.tensor([14.285357442943784], device=device),
                                           'waveform': torch.tensor([[0., 1000.0, 0.]], device=device)}},
                   (0, 2): {'operation': 'fm_saw',
                            'parameters': {'fm_active': torch.tensor([[-1000.0]], device=device),
                                           'active': torch.tensor([[-1000.0]], device=device),
                                           'amp_c': torch.tensor([0.6187255599871848], device=device),
                                           'freq_c': torch.tensor([349.22823143300377], device=device),
                                           'mod_index': torch.tensor([0.02403950683025824], device=device)}}}

target_params_01 = normalizer.normalize(target_param_dict)
print(f"target parameters 0-1: \n{target_params_01}")

plt.figure(figsize=(25, 10))
plt.title("target signal")
plt.plot(target_signal.detach().numpy().squeeze())
plt.show()

decoder_net.apply_params(target_params_01)

parameters_to_freeze = {(1, 1): {'operation': 'lfo',
                                      'parameters': ['freq', 'waveform', 'active']},
                        (0, 2): {'operation': 'fm_saw',
                                 'parameters': ['active', 'fm_active', 'amp_c', 'mod_index']}}

decoder_net.freeze_params(parameters_to_freeze)


predicted_params_01 = decoder_net()
print(f"decoder_net output: \n {predicted_params_01}")

predicted_params_full_range = normalizer.denormalize(predicted_params_01)
print(f"decoder_net output full range: \n {predicted_params_full_range}")

synth.update_cells_from_dict(predicted_params_full_range)
generated_target_signal, _ = synth.generate_signal(signal_duration=1, batch_size=1)

plt.figure(figsize=(25, 10))
plt.title("decoder_net output signal")
plt.plot(generated_target_signal.cpu().detach().numpy().squeeze())
plt.show()

In [ ]:
from tqdm import tqdm

cfg.multi_spectral_loss_spec_type = 'SPECTROGRAM'
loss_handler = spectral_loss.SpectralLoss(loss_type=cfg.loss.spec_loss_type,
                                          loss_preset="mag_logmag",
                                          synth_constants=synth_constants, device=device)

fm_freq_vals = np.linspace(0, 2000, 20000)

losses = []
for freq_val in tqdm(fm_freq_vals):
    update_params = copy.deepcopy(target_param_dict)

    update_params[(0, 2)]['parameters'].update({'freq_c': freq_val})

    synth.update_cells_from_dict(update_params)
    generated_signal, _ = synth.generate_signal(signal_duration=1)

    target_signal_unsqueezed = target_signal.to(device).unsqueeze(dim=0)
    loss_val, _, _ = loss_handler.call(target_signal_unsqueezed, generated_signal, step=0, return_spectrogram=False)

    losses.append(loss_val.detach().cpu().numpy().item())

plt.figure(figsize=(25, 5))
plt.plot(fm_freq_vals, losses)

In [ ]:
num_epochs = 200
starting_frequency = 0.155
decoder_net.apply_params_partial({(0, 2):
                                     {'operation': 'fm_saw',
                                      'parameters': {'freq_c': starting_frequency}
                                     }
                                 })

base_lr = 1e-3
optimizer = torch.optim.Adamax(decoder_net.parameters())

train_res = []
for e in range(num_epochs):

    predicted_params_01 = decoder_net.forward()
    # print(f"decoder_net output: \n {predicted_params_01}")

    predicted_params_full_range = normalizer.denormalize(predicted_params_01)
    predicted_freq = predicted_params_full_range[(0, 2)]['parameters']['freq_c']
    # print(f"decoder_net output full range: \n {predicted_params_full_range}")

    synth.update_cells_from_dict(predicted_params_full_range)
    predicted_signal, _ = synth.generate_signal(signal_duration=1)

    # plt.figure(figsize=(25, 10))
    # plt.plot(target_signal.detach().numpy().squeeze())
    # plt.plot(predicted_signal.cpu().detach().numpy().squeeze())
    # plt.show()

    target_signal_unsqueezed = target_signal.to(device).unsqueeze(dim=0)
    loss, _, _ = loss_handler.call(target_signal_unsqueezed, predicted_signal, step=e)
    # print(f"loss: {loss.item()}")

    # loss = params_loss_handler(predicted_params_01[(0, 2)]['parameters']['freq_c'], torch.tensor(0.15).cuda())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_res.append((e, predicted_freq.item(), loss.item(), decoder_net.parameters_dict[decoder_net.get_key((0, 2), 'fm_saw', 'freq_c')].weight.grad))
    print(decoder_net.parameters_dict[decoder_net.get_key((0, 2), 'fm_saw', 'freq_c')].weight.grad)


In [ ]:
%matplotlib notebook
from matplotlib import animation

fig, ax = plt.subplots(figsize=(25, 5))
l1, = ax.plot(fm_freq_vals, losses, 'o-', label='loss surface', markevery=[-1])
l2, = ax.plot([], [], 'o-', label='training progress')
ax.legend(loc='center right')
# ax.set_xlim(0,100)
# ax.set_ylim(0,1)


def animate(i):
    xi = [train_res[j][1] for j in range(i)]
    yi = [train_res[j][2] for j in range(i)]
    l2.set_data(xi, yi)
    return (l2)

animation.FuncAnimation(fig, animate, frames=num_epochs, interval=500)